# 00_extraccion.ipynb

Este notebook extrae los datos raw desde Overpass API y Google Places API.

## 1. Configuración

In [ ]:
import os
import pandas as pd
import requests

OVERPASS_URL = os.getenv('OVERPASS_API_URL', 'http://overpass-api.de/api/interpreter')
OUTPUT_CSV = 'googlemaps/data/locales_todos.csv'


## 2. Extracción con Overpass API

In [ ]:
query = """
[out:json][timeout:60];
area["name"="Madrid"]->.searchArea;
(
  node["shop"](area.searchArea);
  node["amenity"](area.searchArea);
);
out body;
"""

resp = requests.post(OVERPASS_URL, data={'data': query})
data = resp.json()

records = []
for elem in data['elements']:
    tags = elem.get('tags', {})
    records.append({
        'id_local': elem['id'],
        'latitud': elem.get('lat'),
        'longitud': elem.get('lon'),
        'nombre': tags.get('name', ''),
        'categoria_tags': tags.get('shop') or tags.get('amenity') or 'desconocido'
    })

df_raw = pd.DataFrame(records)
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
df_raw.to_csv(OUTPUT_CSV, index=False)
print(f"Guardados {len(df_raw)} registros en {OUTPUT_CSV}")